In [4]:
# Imports 
from pdf2image import convert_from_path
import pytesseract

import os 
import sys
from tqdm.autonotebook import tqdm

# Going to use MarkItDown because it translates PDFs to LLM readable text better
from markitdown import MarkItDown

# Using HuggingFace 
from huggingface_hub import InferenceClient, whoami
whoami()


{'type': 'user',
 'id': '65ea667d5e5fc34fe5662e9a',
 'name': 'Jefflinbc',
 'fullname': 'Jeff ',
 'isPro': False,
 'avatarUrl': '/avatars/80695d5e468d9384fe54eae13ee58aa5.svg',
 'orgs': [{'type': 'org',
   'id': '649487b364bfbfa4fa60c1bb',
   'name': 'UCSF',
   'fullname': 'University of California, San Francisco',
   'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/62d125ab6c1592eeecdaf4a1/-669Yw7FPbhSREO8XxjZm.jpeg',
   'isEnterprise': False}],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'general',
   'role': 'fineGrained',
   'createdAt': '2025-03-16T08:00:56.012Z',
   'fineGrained': {'canReadGatedRepos': True,
    'global': ['inference.serverless.write'],
    'scoped': [{'entity': {'_id': '65ea667d5e5fc34fe5662e9a',
       'type': 'user',
       'name': 'Jefflinbc'},
      'permissions': ['repo.content.read',
       'repo.write',
       'inference.endpoints.infer.write',
       'inference.endpoints.write']}]}}}}

In [ ]:
# def extract_text_from_pdf(pdf_path):
#     text = ""
#     with open(pdf_path, 'rb') as pdf_file:
#         pdf_reader = PyPDF2.PdfReader(pdf_file)
#         for page_num in range(len(pdf_reader.pages)):
#             page = pdf_reader.pages[page_num]
#             text += page.extract_text()
#     return text


# Simpler text extractor
def extract_text_from_pdf(pdf_path):
    md = MarkItDown(enable_plugins=False, docintel_endpoint="<document_intelligence_endpoint>")
    result = md.convert(pdf_path)
    return result.text_content

# Since some PDFs can be scans, convert them to image and user OCR to read text
def ocr_text_from_pdf(pdf_path):
    pages = convert_from_path(pdf_path, 300)
    text = ''
    for page in pages:
        # OCR :) 
        text += pytesseract.image_to_string(page)
    return text

In [ ]:
# Main block
folder = 'SBA_PDF'
pdf_text = []
for name in tqdm(os.listdir(folder)):
    pdf = folder + '/' + name
    text = extract_text_from_pdf(pdf)

    if len(text) < 1: 
        pdf_text.append(ocr_text_from_pdf(pdf)) 
    else: 
        pdf_text.append(text)

satralizumab = pdf_text[2]

# Inference on all files
# client = InferenceClient(model= "Qwen/QwQ-32B")
# inference_outputs = []
# for sba in pdf_text:
#     print(sba)
#     inference_outputs.append(sba_text_inference(client, sba))
    

  8%|▊         | 1/13 [00:01<00:13,  1.09s/it]

9.pdf SBA_PDF/9.


 15%|█▌        | 2/13 [00:02<00:10,  1.01it/s]

8.pdf SBA_PDF/8.


 23%|██▎       | 3/13 [00:03<00:12,  1.24s/it]

...

1.pdf SBA_PDF/1.


 92%|█████████▏| 12/13 [01:20<00:06,  6.44s/it]

3.pdf SBA_PDF/3.


100%|██████████| 13/13 [01:27<00:00,  6.71s/it]

2.pdf SBA_PDF/2.


In [65]:
satralizumab = pdf_text[2]

prompt = """

			You are an FDA reviewer and expert pharmacologist.

			Your job is to extract only the required information from the SBA document provided by the user.

			You must answer the following questions strictly in the format below. 
			DO NOT provide extra commentary, summaries, or interpretations. Just extract what is asked.

			### RESPONSE FORMAT (MUST FOLLOW THIS EXACT TEMPLATE):

			1. Exposure-safety analyses:
			<Your answer here>

			2. Exposure-efficacy analyses:
			<Your answer here>

			3. Post-marketing requirements or commitments:
			<Your answer here>

			4. Maximum Tolerated Dose (MTD):
			<Your answer here>

			Repeat: DO NOT summarize or add additional content outside this format.	
   
			Below is the SBA document to summarize: 

"""

In [66]:
client = InferenceClient(provider="nebius")

messages = [
	{
		"role": "user",
		"content": f"{prompt}" + f"\n {satralizumab}"
  
	}	
]

result = client.chat.completions.create(
	messages = messages,
	model="microsoft/Phi-3-medium-128k-instruct",
	max_tokens = 500,
)
print(result)

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content=' 1. Exposure-safety analyses: The clinical pharmacology review supports the effectiveness of satralizumab with pivotal Phase 3 clinical trial results showing a significant reduction in the risk of adjudicated relapse. No dose adjustments are needed based on intrinsic factors such as age, race, sex, bodyweight, renal or hepatic impairment, drug/transporter interactions. However, the development of anti-drug antibodies (ADAs) has been observed, which is correlated to higher bodyweight and lower satralizumab exposure, although its impact on efficacy is inconclusive and not clinically relevant to safety.\n\n2. Exposure-efficacy analyses: The clinical pharmacology program provides supportive evidence of effectiveness. The pivotal Phase 3 clinical trials demonstrate a significant reduction in the risk of an adjudicated relapse with treatment

In [68]:
result.choices[0].message['content']

' 1. Exposure-safety analyses: The clinical pharmacology review supports the effectiveness of satralizumab with pivotal Phase 3 clinical trial results showing a significant reduction in the risk of adjudicated relapse. No dose adjustments are needed based on intrinsic factors such as age, race, sex, bodyweight, renal or hepatic impairment, drug/transporter interactions. However, the development of anti-drug antibodies (ADAs) has been observed, which is correlated to higher bodyweight and lower satralizumab exposure, although its impact on efficacy is inconclusive and not clinically relevant to safety.\n\n2. Exposure-efficacy analyses: The clinical pharmacology program provides supportive evidence of effectiveness. The pivotal Phase 3 clinical trials demonstrate a significant reduction in the risk of an adjudicated relapse with treatment with satralizumab. The recommended dose regimen is supported by the results of the trials, with no alternative dosing regimen and management strategy i

In [69]:
client = InferenceClient(provider="nebius")

messages = [
	{
		"role": "user",
		"content": f"Summarize the information (i - iv above) you extracted from the SBA assigned to you in 200 words or less: {result.choices[0].message['content']}"
  
	}	
]

result = client.chat.completions.create(
	messages = messages,
	model="microsoft/Phi-3-medium-128k-instruct",
	max_tokens = 500,
)